# Кибериммунная автономность$\\$Создание конструктивно защищённого автономного наземного транспортного средства$\\$Модуль 2

## О документе

Версия 1.03

Модуль 2 для регионального этапа соревнований по кибериммунной автономности

## Подготовка к работе c модулем 2

Перенесите в блок ниже ваши реализации классов коммуникационного шлюза (CommunicationGateway), систему навигации (NavigationSystem), управления (ControlSystem)

In [1]:
# ваш код

Если у вас настроена и работает СУПА, установите в True значение переменной afcs_present

In [2]:
afcs_present = True

В этом модуле изменим идентификатор машинки:

In [3]:
car_id = "m2" 

### Модуль 2. Отправка маршрутного здания и следование по маршруту

В рамках этого модуля отрабатывается только автономность. 
Никаких киберпрепятствий пока не будет. 

Что нужно сделать по пунктам:
1. Для заданных начальных и конечных точек создать маршрут, содержащий не менее пяти промежуточных путевых точек вручную или с использованием таких инструментов как APMPlanner, MissionPlanner. <br>Содержимое wpl файла нужно вставить в соответствующий кодовый блок блокнота, запустить симуляцию и убедиться, что машинка успешно проходит этот маршрут.
2. Задать скоростные ограничения в виде массива элементов GeoSpecificSpeedLimit
3. Запустить симуляцию


Формат WPL файла описан на этой странице: 

https://mavlink.io/en/file_formats/#mission_plain_text_file

Пример содержимого файла с текстовым описанием маршрута

In [ ]:
wpl_file_content =  """QGC WPL 110
0	1	0	16	0	5	0	0	59.9391061791295741	30.3156781196594238	0	1
1	0	3	16	0	5	0	0	59.938515001968419	30.312201976776123	0	1
2	0	3	16	0	5	0	0	59.9421371401196978	30.3066229820251465	0	1
3	0	3	16	0	5	0	0	59.9410946067827126	30.3039407730102539	0	1
4	0	3	16	0	5	0	0	59.9403744872963955	30.302116870880127	0	1
5	0	3	16	0	5	0	0	59.9412316426749499	30.3011056780815125	0	1
6	0	3	16	0	5	0	0	59.9411147591553188	30.3006818890571594	0	1
"""


Для выполнения задания в блоке ниже  
1. замените имя файла с маршрутом (на свой), вот эту строку:
   ```python
   wpl_file = "module2.wpl"
   ```
2. измените список скоростных ограничений, вот этот массив:
   ```python
   speed_limits = [
    GeoSpecificSpeedLimit(...
    ```
   
Учтите, что вам нужно доставить груз как можно быстрее, но при этом не нарушить правила дорожного движения. 

Ограничения скорости
- населённый пункт - 60 км/ч
- пешеходная зона - 20 км/ч
- магистраль - 110 км/ч

Составьте маршрут доставки так, чтобы машинка доехала за минимальное время. Для определения оптимального маршрута допустимо пользоваться сторонними сервисами (например, Яндекс картами).

In [ ]:
# создадим wpl файл и запишем туда маршрутное задание
wpl_file = "module2.wpl"

**Примечание**:
После создания собственного маршрута удалите перезапись содержимого, чтобы не потерять свои наработки!


In [7]:
with open(wpl_file, "w") as f:
    f.write(wpl_file_content)

**Напоминание**: не забудьте изменить скоростные ограничения (speed_limits) при задании маршрутного задания!

In [8]:
from src.wpl_parser import WPLParser

parser = WPLParser(wpl_file)    
points = parser.parse()
print(points)

from src.mission_type import GeoSpecificSpeedLimit
speed_limits = [
    GeoSpecificSpeedLimit(0, 30),
    GeoSpecificSpeedLimit(1, 60),
    GeoSpecificSpeedLimit(5, 20),
]

from src.mission_planner import Mission

home = points[0]
mission = Mission(home=home, waypoints=points,speed_limits=speed_limits, armed=True)

[Point(59.939106179129574, 30.315678119659424, 0.0), Point(59.93851500196842, 30.312201976776123, 0.0), Point(59.9421371401197, 30.306622982025146, 0.0), Point(59.94109460678271, 30.303940773010254, 0.0), Point(59.940374487296396, 30.302116870880127, 0.0), Point(59.94123164267495, 30.301105678081512, 0.0), Point(59.94111475915532, 30.30068188905716, 0.0)]


Запускаем симуляцию с новым маршрутным заданием

In [9]:
from time import sleep
from geopy import Point as GeoPoint


from src.queues_dir import QueuesDirectory
from src.servos import Servos
from src.sitl import SITL
from src.cargo_bay import CargoBay
from src.mission_planner import MissionPlanner
from src.config import LOG_ERROR, LOG_INFO
from src.mission_planner_mqtt import MissionSender
from src.sitl_mqtt import TelemetrySender
from src.system_wrapper import SystemComponentsContainer


# координата текущего положения машинки
home = GeoPoint(latitude=59.939032, longitude=30.315827) 


# каталог очередей для передачи сообщений между блоками
queues_dir = QueuesDirectory() 

if afcs_present:
    mission_sender = MissionSender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)
    telemetry_sender = TelemetrySender(
        queues_dir=queues_dir, client_id=car_id, log_level=LOG_ERROR)

mission_planner = MissionPlanner(
    queues_dir, afcs_present=afcs_present, mission=mission)

sitl = SITL(
    queues_dir=queues_dir, position=home,
    car_id=car_id, post_telemetry=afcs_present, log_level=LOG_ERROR)


communication_gateway = CommunicationGateway(
    queues_dir=queues_dir, log_level=LOG_ERROR)
control_system = ControlSystem(queues_dir=queues_dir, log_level=LOG_INFO)

navigation_system = NavigationSystem(
    queues_dir=queues_dir, log_level=LOG_ERROR)

servos = Servos(queues_dir=queues_dir, log_level=LOG_ERROR)
cargo_bay = CargoBay(queues_dir=queues_dir, log_level=LOG_INFO)

# у нас получилось довольно много блоков, используем класс SystemComponentsContainer
# для упрощения рутинной работы с ними - таким образом мы собираем все блоки машинки в одном "кузове"
system_components = SystemComponentsContainer(
    components=[
        mission_sender,
        telemetry_sender,
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system
    ] if afcs_present else [
        sitl,
        mission_planner,
        navigation_system,
        servos,
        cargo_bay,
        communication_gateway,
        control_system
    ])

system_components.start()

# ограничение поездки по времени
# параметр sleep - время в секундах,
# настройте этот параметр так, чтобы ваша машинка завершила маршрут
# в случае превышения времени выполнения ячейки на более чем 10 секунд от заданного, 
# допустимо перезапустить вычислительное ядро и повторно выполнить весь блокнот, штрафные очки за это не начисляются
# при условии, что повторный запуск закончился успешно
sleep(100)

# останавливаем все компоненты
system_components.stop()

# удалим все созданные компоненты
system_components.clean()

[ИНФО][QUEUES] создан каталог очередей
[ИНФО][QUEUES] регистрируем очередь planner.mqtt
[ИНФО][QUEUES] регистрируем очередь sitl.mqtt
[ИНФО][QUEUES] регистрируем очередь planner
[ИНФО][MISSION PLANNER] создана система планирования заданий
[ИНФО][QUEUES] регистрируем очередь sitl
[ИНФО][QUEUES] регистрируем очередь communication
[ИНФО][QUEUES] регистрируем очередь control
[ИНФО][CONTROL] создана система управления
[ИНФО][QUEUES] регистрируем очередь navigation
[ИНФО][QUEUES] регистрируем очередь servos
[ИНФО][QUEUES] регистрируем очередь cargo
[ИНФО][CARGO] создан компонент грузового отсека, отсек заблокирован
[ИНФО][CARGO] старт блока грузового отсека
[ИНФО][CONTROL] старт системы управления
[ИНФО][MISSION PLANNER] старт системы планирования заданий
[ИНФО][MISSION PLANNER] запрошена новая задача, отправляем получателям
[ИНФО][MISSION PLANNER] новая задача отправлена в коммуникационный шлюз


[ИНФО][CONTROL] установлена новая задача, начинаем следовать по маршруту, текущее время 10:23:40.127052
[ИНФО][CARGO] заблокировать грузовой отсек
[ИНФО][CARGO] грузовой отсек заблокирован
[ИНФО][CONTROL] новая скорость 30 (была 0)
[ИНФО][CONTROL] новое направление 254 (было 0)
[ИНФО][CONTROL] сегмент пройден
[ИНФО][CONTROL] новая скорость 60 (была 30)
[ИНФО][CONTROL] новое направление 322 (было 254)
[ИНФО][CONTROL] сегмент пройден
[ИНФО][CONTROL] новое направление 233 (было 322)
[ИНФО][CONTROL] новое направление 230 (было 233)
[ИНФО][CONTROL] новое направление 228 (было 230)
[ИНФО][CONTROL] новое направление 227 (было 228)
[ИНФО][CONTROL] новое направление 228 (было 227)
[ИНФО][CONTROL] сегмент пройден
[ИНФО][CONTROL] новое направление 231 (было 228)
[ИНФО][CONTROL] новое направление 232 (было 231)
[ИНФО][CONTROL] сегмент пройден
[ИНФО][CONTROL] новое направление 327 (было 232)
[ИНФО][CONTROL] новое направление 331 (было 327)
[ИНФО][CONTROL] новое направление 336 (было 331)
[ИНФО][CON

На этом модуль 2 завершён, надеемся, у вас всё получилось! 
На что можно ещё обратить вниманием - можно ли как-то улучшить маршрут для ускорения доставки? (без нарушения ПДД) - например, использовать более скоростные трассы или срезать какие-то участки.

Если идей быстрых улучшений нет, то пришло время перейти к самому интересному модулю и познакомиться с киберпрепятствиями!

Для работы над модулем 3 задания откройте блокнот cyberimmunity--autonomous-car-m3.